#### Import the required packages with their customary aliases

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
covidzip_df =pd.read_csv('../data/COVID_CountByZipPerDate 03292021.csv')
covidzip_df.head()

,Specimen Collection Date,Zip,Cases
0,3/6/2020,37217.0,1
1,3/10/2020,37214.0,1
2,3/11/2020,37013.0,1
3,3/11/2020,37204.0,2
4,3/11/2020,37205.0,1


In [10]:
covidzip_df.info

<bound method DataFrame.info of      Specimen Collection Date      Zip  Cases
0                    3/6/2020  37217.0      1
1                   3/10/2020  37214.0      1
2                   3/11/2020  37013.0      1
3                   3/11/2020  37204.0      2
4                   3/11/2020  37205.0      1
...                       ...      ...    ...
8827                3/27/2021  37210.0      1
8828                3/27/2021  37211.0      1
8829                3/27/2021  37212.0      1
8830                3/27/2021  37214.0      2
8831                3/27/2021  37221.0      2

[8832 rows x 3 columns]>

In [12]:
covidzip_df.dtypes

Specimen Collection Date     object
Zip                         float64
Cases                         int64
dtype: object

In [23]:
covidzip_df['Zip'] = covidzip_df['Zip'].astype('Int64')
covidzip_df.dtypes

Specimen Collection Date    object
Zip                          Int64
Cases                        int64
dtype: object

In [17]:
covidzip_df.tail(20)

,Specimen Collection Date,Zip,Cases
8812,3/26/2021,37216.0,1
8813,3/26/2021,37217.0,2
8814,3/26/2021,37218.0,1
8815,3/26/2021,37220.0,3
8816,3/26/2021,37221.0,7
8817,3/26/2021,37228.0,1
8818,3/27/2021,NaN,1
8819,3/27/2021,37013.0,1
8820,3/27/2021,37072.0,1
8821,3/27/2021,37115.0,1


In [25]:
covidzip_df.info

<bound method DataFrame.info of      Specimen Collection Date    Zip  Cases
0                    3/6/2020  37217      1
1                   3/10/2020  37214      1
2                   3/11/2020  37013      1
3                   3/11/2020  37204      2
4                   3/11/2020  37205      1
...                       ...    ...    ...
8827                3/27/2021  37210      1
8828                3/27/2021  37211      1
8829                3/27/2021  37212      1
8830                3/27/2021  37214      2
8831                3/27/2021  37221      2

[8832 rows x 3 columns]>

In [24]:
v_protocol_df =pd.read_csv('../data/vaccine_with_protocol.csv')
v_protocol_df.head()

,vacc_date,zipcode,count_given,dose_nbr,protocol
0,2020-12-12,37215.0,1.0,1.0,two-shot
1,2020-12-14,37205.0,1.0,2.0,two-shot
2,2020-12-14,37215.0,1.0,1.0,two-shot
3,2020-12-15,37221.0,1.0,1.0,two-shot
4,2020-12-16,37215.0,1.0,1.0,two-shot


In [27]:
v_protocol_df.dtypes

vacc_date       object
zipcode        float64
count_given    float64
dose_nbr       float64
protocol        object
dtype: object

In [28]:
v_protocol_df['zipcode'] = v_protocol_df['zipcode'].astype('Int64')
v_protocol_df.dtypes

vacc_date       object
zipcode          Int64
count_given    float64
dose_nbr       float64
protocol        object
dtype: object